In [ ]:
import os, sys, json
import numpy as np
import torch
sys.path.append('..')

from src.config import ModelParams, TrainConfig
from src.deqn import PolicyNetwork, Trainer, simulate_paths
from src.io_utils import make_run_dir, save_run_metadata, save_selected_run, pack_config, save_torch, save_csv, save_json, save_npz, ensure_dir
from src.metrics import residual_quality

# ---------- config ----------
ARTIFACTS_ROOT = os.path.join("..", "artifacts")
params = ModelParams(device="cpu", dtype=torch.float32)
cfg = TrainConfig.full(seed=0)

run_dir = make_run_dir(ARTIFACTS_ROOT, "discretion", tag=cfg.mode, seed=cfg.seed)
save_run_metadata(run_dir, pack_config(params, cfg, extra={"policy":"discretion"}))
print("Run dir:", run_dir)


In [ ]:
rbar = None  # only used for mod_taylor


In [ ]:

# ---------- model ----------
d_in, d_out = 5, 11
net = PolicyNetwork(d_in, d_out, hidden=cfg.hidden_layers, activation=cfg.activation)

trainer = Trainer(
    params=params,
    cfg=cfg,
    policy="discretion",
    net=net,
    rbar_by_regime=rbar if "discretion"=="mod_taylor" else None,
)



In [ ]:

# ---------- train ----------
losses = trainer.train(
    B=4096,
    commitment_sss=None
)

# save weights and log
save_torch(os.path.join(run_dir, "weights.pt"), trainer.net.state_dict())
import pandas as pd
df = pd.DataFrame({"iter": np.arange(len(losses)), "loss": losses})
save_csv(os.path.join(run_dir, "train_log.csv"), df)

# quality on a fresh validation batch from the replay sampler
with torch.enable_grad():
    x_val = trainer.replay.sample(int(cfg.val_size))
    resid = trainer._residuals(x_val).detach().cpu().numpy()
q = residual_quality(resid, tol=getattr(cfg, "report_tol", 1e-3))
save_json(os.path.join(run_dir, "train_quality.json"), q)
print("Train quality:", q)

# optional: mark this run as selected for results notebook
save_selected_run(ARTIFACTS_ROOT, trainer.policy, run_dir)

# ---------- simulate ergodic paths ----------
x0 = trainer.simulate_initial_state(512, commitment_sss=None)
sim = simulate_paths(
    params=params,
    policy=trainer.policy,
    net=trainer.net,
    T=30000,
    burn_in=3000,
    x0=x0,
    rbar_by_regime=rbar if trainer.policy=="mod_taylor" else None,
    compute_implied_i=False,
    gh_n=3,
    thin=20,
)
save_npz(os.path.join(run_dir, "sim_paths.npz"), **sim)
print("Saved sim_paths to:", os.path.join(run_dir, "sim_paths.npz"))


In [ ]:
# ---------- SSS from trained policy (paper-faithful) ----------
from src.steady_states import solve_discretion_sss_from_policy
disc_sss = solve_discretion_sss_from_policy(params, trainer.net)
save_json(os.path.join(run_dir, 'sss_policy_fixed_point.json'), {'policy':'discretion','by_regime': disc_sss.by_regime})
print('=== DISCRETION SSS (policy fixed point, by regime) ===')
for _s in sorted(disc_sss.by_regime.keys()):
    print(f'Regime {_s}:')
    for _k,_v in disc_sss.by_regime[_s].items():
        print(f'{_k:>20}: {_v}')


In [ ]:
# ---------- Sanity checks (fixed point + residuals) ----------
from src.sanity_checks import fixed_point_check, residuals_check
fp = fixed_point_check(params, trainer.net, policy='discretion', sss_by_regime=disc_sss.by_regime)
rc = residuals_check(params, trainer.net, policy='discretion', sss_by_regime=disc_sss.by_regime)
print('Fixed point max |x_next-x| by regime:', {k:v.max_abs_state_diff for k,v in fp.items()})
print('Residual check max |res| by regime:', {k:v.max_abs_residual for k,v in rc.items()})
print('Residual keys:', list(next(iter(rc.values())).residuals.keys()))


In [ ]:
# ---------- Save sanity checks ----------
save_json(os.path.join(run_dir, 'sanity_checks.json'), {
    'policy': 'discretion',
    'fixed_point_max_abs_state_diff': {int(k): float(v.max_abs_state_diff) for k,v in fp.items()},
    'residual_max_abs': {int(k): float(v.max_abs_residual) for k,v in rc.items()},
    'residuals_by_regime': {int(k): {kk: float(vv) for kk,vv in v.residuals.items()} for k,v in rc.items()},
})